# Reich Moore

In [4]:
print(21.0266269/2.119539e+1)
print(21.0023219/2.117089e+1)
print(20.9659142/2.113419e+1)
print(20.9295857/2.109757e+1)
print(20.8812338/2.104883e+1)
print(20.8450542/2.101236e+1)
print(20.8330009/2.100021e+1)
print(20.7848871/2.095171e+1)
print(20.7368923/2.090333e+1)


0.99203774500021
0.9920377414459193
0.9920377454730936
0.9920377417873243
0.9920377427153909
0.9920377434995402
0.9920377415273466
0.9920377429813605
0.9920377423118709


In [ ]:
import numpy as np
from ENDFtk.tree import Tape
from ENDFtk.MF2.MT151 import ResonanceRange
from ENDFtk.MF2.MT151 import Isotope
from ENDFtk.MF2.MT151 import Section
import h5py, os
import numpy as np
import importlib
import CovarianceResonance
importlib.reload(CovarianceResonance)
from CovarianceResonance import ResonanceCovariance as RC

# Read the ENDF file
# tapeMBCnURR = Tape.from_file('/home/sole-pie01/work/ndlib/endfb8-neutron/n-026_Fe_055.endf')    # OK
tapeRMCnURR = Tape.from_file('/home/sole-pie01/work/ndlib/tendl17-neutron/n-U238-s20.tendl17.txt')# OK
# tapeRM = Tape.from_file('/home/sole-pie01/work/ndlib/endfb8-neutron/n-094_Pu_239.endf')         # OK
# tapeRMLC = Tape.from_file('/home/sole-pie01/work/ndlib/endfb8-neutron/n-029_Cu_063.endf')       # OK

tape = tapeRMCnURR
mf2 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
mf32 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()

# Get the resonance ranges from MF2 and MF32
mf2_resonance_ranges = mf2.isotopes[0].resonance_ranges.to_list()
mf32_resonance_ranges = mf32.isotopes[0].resonance_ranges.to_list()

NER = 1
covariance_obj = RC.from_resonance_range(mf32_resonance_ranges[NER], mf2_resonance_ranges, NER)
# Access the covariance matrix and parameters
covariance_matrix = covariance_obj.covariance_matrix
parameters = covariance_obj.parameters
# Use the data as needed
print(f"\nCovariance matrix for NER={NER}:\n{covariance_matrix[:4, :4]}")
covariance_obj.print_parameters()



Covariance matrix for NER=1:
[[0.0001 0.     0.     0.    ]
 [0.     0.01   0.     0.    ]
 [0.     0.     0.0001 0.    ]
 [0.     0.     0.     0.01  ]]
Averaged Breit-Wigner Parameters for NER=1:
(L=0, J=0.5):
  D: [21.19539 21.17089 21.13419 21.09757 21.04883 21.01236 21.00021 20.95171
 20.90333 20.89125 20.85506 20.80691 20.77087 20.71095 20.66314 20.59165
 20.52041 20.47307 20.41404 20.35519 20.30824 20.23803 20.19136 20.12156
 20.05202 20.0058  20.00089 20.00087 20.00084 20.00081 20.00072 20.00053
 20.00026 19.99979 19.99933 19.99749 19.99426 19.98964 19.98365 19.97812
 19.9689  19.95968 19.94817 19.93667 19.91368 19.89072 19.86779 19.82201
 19.77634 19.73078 19.68532 19.63998 19.54962 19.5046  19.41489 19.3256
 19.2146  19.10426 18.99458 18.88555 18.77718 18.66945 18.56237 18.45593
 18.35012 18.24494 18.14039 18.03646 17.93316 17.83047 17.72839 17.62692
 17.42578 17.22704 17.03065 16.83659 16.64484 16.45535 16.26811 16.08309
 15.90025 15.75554 15.75523]
  GN: [0.00216681 0.0021

In [ ]:
from ENDFtk.MF2.MT151 import ReichMooreLValue
from ENDFtk.MF2.MT151 import ReichMoore

MATMF2MT151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()

isotopeIndex = 0 # Always zero
rangeIndex = 0 # User defined
RANGE = MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex]

# We must build a ReichMoore object with our samples and then replace the one from the original tape, then duplicate the tape.
# In this example I fill lgroups with the same data than the original file. The section can not be modified directly (read access only) but we can create a pseudo
# section with modified value, and replace it.

###########################
####  BLOCK TO MODIFY  ####
###########################

# Our HDF5 object will contain AP, parameters associated to a L values like APL, parameters associated with L and resonanceIndex : ER, GN, GG, GFA, GFB 

dummy_lgroups = []
for iL, lgroup in enumerate(RANGE.parameters.l_values.to_list()):
    # ReichMooreLValue takes 3 integers [AWRI, APL, L] and lists of parameters
    dummy_ER  = lgroup.ER  
    dummy_GN  = lgroup.GN 
    dummy_GG  = lgroup.GG 
    dummy_GFA = lgroup.GFA 
    dummy_GFB = lgroup.GFB 
    dummy_APL = lgroup.APL 
    # At this point, replace the dummies with our samples, if not present then the original value are assumed to stay in place.
    # Example : we have a random energy associated to L=1 indexReso=85
    #           when iL=1 we set dummy_ER[85] = sample_ER[L=1][indexReso=85]  
    # Think of an efficient way to replace those sampled parameters
    dummy_lgroups.append( ReichMooreLValue( lgroup.AWRI, lgroup.APL, lgroup.L,
                                            # 6 lists of parameters, each with NRS resonances (lgroup.NRS)
                                            # Each resonance has :
                                            dummy_ER, # an energy,
                                            lgroup.AJ, # a spin, as it cannot be varied we set the original one
                                            dummy_GN, # a neutron width,
                                            dummy_GG, # a capture width,
                                            dummy_GFA, # a first fission width,
                                            dummy_GFB) )  # a second fission width,

dummy_ap = RANGE.parameters.AP # If AP has been sampled (present ot not null), then replace it
# ReichMoore takes a list [spin, ap, lad, nlsc, lvalues], all integers except lad (boolean) and lvalues defined previously

dummy_parameters = ReichMoore( spin = RANGE.parameters.SPI, ap = dummy_ap, lad = RANGE.parameters.LAD, nlsc = RANGE.parameters.NLSC,
                               lvalues = dummy_lgroups )

###########################
###########################
###########################

# Then stuff the section and replace
isotopes = [ Isotope( zai = MATMF2MT151.isotopes[isotopeIndex].ZAI, 
                      abn = MATMF2MT151.isotopes[isotopeIndex].ABN, 
                      lfw = MATMF2MT151.isotopes[isotopeIndex].LFW,
                      ranges = [ ResonanceRange( el = RANGE.EL, 
                                                 eh = RANGE.EH,
                                                 naps = RANGE.NAPS, 
                                                 parameters = dummy_parameters ) ] ) ]
dummySection = Section( zaid = MATMF2MT151.ZA, awr = MATMF2MT151.AWR, isotopes = isotopes )

tape.MAT(tape.material_numbers[0]).MF(2).insert_or_replace( dummySection )

# Create a dummy duplicate
tape.to_file("toto.endf")

# Unresolved Energy Dependent (LFW=0, LRF=2)

In [11]:
from ENDFtk.MF2.MT151 import UnresolvedEnergyDependent
from ENDFtk.MF2.MT151 import UnresolvedEnergyDependentLValue
from ENDFtk.MF2.MT151 import UnresolvedEnergyDependentJValue

# Read the ENDF file
tapeRMCnURR = Tape.from_file('/home/sole-pie01/work/ndlib/tendl17-neutron/n-U238-s20.tendl17.txt')# OK

tape = tapeRMCnURR
mf2 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
mf32 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()

isotopeIndex = 0 # Always zero
rangeIndex = 1 # User defined
MATMF2MT151 = tape.MAT(tape.material_numbers[0]).MF(2).MT(151).parse()
MATMF32MT151 = tape.MAT(tape.material_numbers[0]).MF(32).MT(151).parse()
RANGE = MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex]

uncertainty = MATMF32MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex].parameters

lgroups = []
for lgroup in RANGE.parameters.l_values.to_list():
    jgroups = []
    for jgroup in lgroup.j_values.to_list():
        # In the Energy Dependant Parameters format, that is when LRF=2, within each lgroup has NJS[L] jgroups and
        # each jgroup has parameters. The parameters D, GN, GG, GF and GX are the only one that may have an uncertainty in MF32 and
        # that will be perturbed. ES, D, GN, GG, GF and GX are vectors, because a jgroup contains energery dependant D,GN,GG,GF,GX, there all associated with an ES.
        # The subtleties is that MF32 provides a relative covariance, it is relative because the uncertainty data is given for a (L,J) couple, not energy dependant,
        # an uncertainty in a (L,J) couple will have to be applied to all energies.
        # Store the relative covariance matrix and all the means of the parameters (energy dependant) and think of a strategy for sampling. 
        # ordered as D,GN,GG,GF,GX depending on MPAR (if MPAR is 3 then D,GN,GG)
        jgroups.append( UnresolvedEnergyDependentJValue(jgroup.AJ, jgroup.AMUN, jgroup.AMUG, jgroup.AMUF, jgroup.AMUX, jgroup.INT,
                                                        jgroup.ES,
                                                        jgroup.D,
                                                        jgroup.GN,
                                                        jgroup.GG,
                                                        jgroup.GF,
                                                        jgroup.GX) )
        
    lgroups.append( UnresolvedEnergyDependentLValue( lgroup.AWRI, lgroup.L, jgroups) )
    

parameters = UnresolvedEnergyDependent(RANGE.parameters.SPI, RANGE.parameters.AP, RANGE.parameters.LSSF, lgroups)

isotopes = [ Isotope( zai = MATMF2MT151.isotopes[isotopeIndex].ZAI, 
                      abn = MATMF2MT151.isotopes[isotopeIndex].ABN, 
                      lfw = MATMF2MT151.isotopes[isotopeIndex].LFW,
                      ranges = [ ResonanceRange( el = RANGE.EL, 
                                                 eh = RANGE.EH,
                                                 naps = RANGE.NAPS, 
                                                 parameters = parameters,
                                                 scattering_radius = RANGE.scattering_radius) ] ) ]

dummySection = Section( zaid = MATMF2MT151.ZA, awr = MATMF2MT151.AWR, isotopes = isotopes )

tape.MAT(tape.material_numbers[0]).MF(2).insert_or_replace( dummySection )
tape.to_file("toto.endf")

In [9]:
RANGE.scattering_radius

# Unresolved Energy Independent

In [ ]:
from ENDFtk.MF2.MT151 import UnresolvedEnergyIndependent
from ENDFtk.MF2.MT151 import UnresolvedEnergyIndependentLValue

isotopeIndex = 0 # Always zero
rangeIndex = 1 # User defined
RANGE = MATMF2MT151.isotopes[isotopeIndex].resonance_ranges[rangeIndex]

RANGE.parameters.l_values[0].j_values[0].AJ

lgroups = []
for lgroup in RANGE.parameters.l_values.to_list():
    AJ = []
    D = []
    AMUN = []
    GN = []
    GG = []
    for jgroup in lgroup.j_values.to_list():
        print(jgroup.D)
        # AJ.append(jgroup.AJ)
        # D.append(jgroup.D)
        # AMUN.append(jgroup.AMUN)
        # GN.append(jgroup.GN)
        # GG.append(jgroup.GG)
        
    lgroups.append( UnresolvedEnergyIndependentLValue( lgroup.AWRI, lgroup.L,
                                                       AJ,
                                                       D,
                                                       AMUN,
                                                       GN,
                                                       GG) )
    

parameters = UnresolvedEnergyIndependent(RANGE.parameters.SPI, RANGE.parameters.AP, RANGE.parameters.LSSF, lgroups)

isotopes = [ Isotope( zai = MATMF2MT151.isotopes[isotopeIndex].ZAI, 
                      abn = MATMF2MT151.isotopes[isotopeIndex].ABN, 
                      lfw = MATMF2MT151.isotopes[isotopeIndex].LFW,
                      ranges = [ ResonanceRange( el = RANGE.EL, 
                                                 eh = RANGE.EH,
                                                 naps = RANGE.NAPS, 
                                                 parameters = parameters ) ] ) ]

dummySection = Section( zaid = MATMF2MT151.ZA, awr = MATMF2MT151.AWR, isotopes = isotopes )

tape.MAT(tape.material_numbers[0]).MF(2).insert_or_replace( dummySection )
tape.to_file("toto.endf")

NameError: name 'MATMF2MT151' is not defined